In [1]:
import os
import cv2
import glob
import keras
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

In [20]:
img_generator = ImageDataGenerator(rotation_range=30,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   rescale=1/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode="nearest")

In [22]:
image_shape = (160,160,3)

In [23]:
train_set = img_generator.flow_from_directory('dataset',
                                              target_size=image_shape[:2],
                                              batch_size=16,
                                              class_mode="binary")

Found 3506 images belonging to 14 classes.


In [24]:
train_set.class_indices

{'aarin ': 0,
 'adithya': 1,
 'aditya': 2,
 'avanish': 3,
 'dev': 4,
 'dhruv_arora': 5,
 'parth_waman': 6,
 'shimpali': 7,
 'shourya': 8,
 'siddharth': 9,
 'tanuj': 10,
 'tarush': 11,
 'utkarsh': 12,
 'vaibhav': 13}

In [26]:
test_set = img_generator.flow_from_directory("dataset",
                                             target_size=image_shape[:2],
                                             batch_size=16,
                                             class_mode="binary")

Found 3506 images belonging to 14 classes.


In [33]:
CNN = Sequential()
CNN.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(150,150,3),  activation='relu'))
CNN.add(MaxPool2D(pool_size=2))
CNN.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
CNN.add(MaxPool2D(pool_size=2))
CNN.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
CNN.add(MaxPool2D(pool_size=2))
CNN.add(Flatten())
CNN.add(Dense(units=200, activation="relu"))
CNN.add(Dropout(0.5))
CNN.add(Dense(units=100, activation="relu"))
CNN.add(Dense(units=14, activation="sigmoid"))

In [34]:
CNN.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [35]:
CNN.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 36, 36, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 34, 34, 64)        36928     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 17, 17, 64)       

In [36]:
CNN.fit(train_set, epochs = 30)

Epoch 1/30


ValueError: in user code:

    File "/Users/skull/PycharmProjects/Face Recognition System/venv/lib/python3.9/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/skull/PycharmProjects/Face Recognition System/venv/lib/python3.9/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/skull/PycharmProjects/Face Recognition System/venv/lib/python3.9/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/skull/PycharmProjects/Face Recognition System/venv/lib/python3.9/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/skull/PycharmProjects/Face Recognition System/venv/lib/python3.9/site-packages/keras/src/engine/training.py", li

In [2]:
faces_embeddings = np.load("faces_embeddings_done_14classes.npz")
Y = faces_embeddings['arr_1']
X = faces_embeddings["arr_0"]

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True, random_state=17)

In [13]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16

# Load the VGG16 model
model = VGG16(weights='imagenet', include_top=False)

In [18]:
# Create a Sequential model
model = tf.keras.Sequential()

# Add the VGG16 model to the Sequential model
model.add(VGG16(weights='imagenet', include_top=False))

# Add a new fully connected layer to the model
model.add(tf.keras.layers.Dense(14, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model on the training data
model.fit(X_train, Y_train, epochs=10)